#Scenario 3: Customer Segmentation Query

#Your Prompt

Using SQL, create a query that segments customers based on purchasing behavior using the following tables:

**user_activity**: user_id, last_login_date, feature_usage_count, account_type

**transactions**: transaction_id, user_id, transaction_date, amount, platform

**user_preferences**: user_id, communication_preference, interface_theme, notification_settings

The query should identify users who have logged in within the last 30 days, filter customers in the top spending group (using top 20% logic or an equivalent spending threshold for smaller datasets), and retrieve their communication preferences, interface themes, and notification settings. Use CTEs or subqueries to keep the query clear and well-structured.

In [14]:
#Generated code

import sqlite3
import pandas as pd

# 1. Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 2. Create mock tables
cursor.executescript('''
CREATE TABLE user_activity (
    user_id INT,
    last_login_date TEXT,
    feature_usage_count INT,
    account_type TEXT
);
CREATE TABLE transactions (
    transaction_id INT,
    user_id INT,
    transaction_date TEXT,
    amount REAL,
    platform TEXT
);
CREATE TABLE user_preferences (
    user_id INT,
    communication_preference TEXT,
    interface_theme TEXT,
    notification_settings TEXT
);
''')

# 3. Insert adjusted mock data (more recent logins and higher spending variety)
cursor.executemany('INSERT INTO user_activity VALUES (?, ?, ?, ?)', [
    (1, '2025-07-20', 10, 'Premium'),
    (2, '2025-07-18', 5, 'Basic'),
    (3, '2025-07-15', 3, 'Premium')
])
cursor.executemany('INSERT INTO transactions VALUES (?, ?, ?, ?, ?)', [
    (1, 1, '2025-07-01', 120.50, 'Web'),
    (2, 1, '2025-07-05', 300.00, 'Mobile'),
    (3, 2, '2025-07-10', 250.00, 'Web'),
    (4, 3, '2025-07-12', 400.00, 'Web')
])
cursor.executemany('INSERT INTO user_preferences VALUES (?, ?, ?, ?)', [
    (1, 'Email', 'Dark', 'Enabled'),
    (2, 'SMS', 'Light', 'Disabled'),
    (3, 'Push', 'Dark', 'Enabled')
])

# 4. SQL Query (simplified top 20% logic for small datasets)
query = '''
WITH active_users AS (
    SELECT user_id
    FROM user_activity
    WHERE DATE(last_login_date) >= DATE('now', '-30 day')
),
user_spending AS (
    SELECT user_id, SUM(amount) AS total_spent
    FROM transactions
    GROUP BY user_id
),
spending_threshold AS (
    SELECT MAX(total_spent) * 0.8 AS top_20_threshold
    FROM user_spending
),
high_value_users AS (
    SELECT us.user_id, us.total_spent
    FROM user_spending us
    CROSS JOIN spending_threshold st
    WHERE us.total_spent >= st.top_20_threshold
)
SELECT ua.user_id,
       ua.account_type,
       hv.total_spent,
       up.communication_preference,
       up.interface_theme,
       up.notification_settings
FROM active_users a
JOIN high_value_users hv ON a.user_id = hv.user_id
JOIN user_activity ua ON a.user_id = ua.user_id
JOIN user_preferences up ON a.user_id = up.user_id;
'''

# 5. Run query and show results
df = pd.read_sql_query(query, conn)
print("Segmented Customers:\n", df)


Segmented Customers:
    user_id account_type  total_spent communication_preference interface_theme  \
0        1      Premium        420.5                    Email            Dark   
1        3      Premium        400.0                     Push            Dark   

  notification_settings  
0               Enabled  
1               Enabled  


## Follow-up Prompts

*  Add average transaction value and transaction count per user to the final result
*   Modify the query so the active user window is, for example, the last X days, instead of fixed at 30 days.

In [15]:
#Final Solution

import sqlite3
import pandas as pd

# Number of days to define active users
days_active = 30  # You can change this value to test different windows

# 1. Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 2. Create mock tables
cursor.executescript('''
CREATE TABLE user_activity (
    user_id INT,
    last_login_date TEXT,
    feature_usage_count INT,
    account_type TEXT
);
CREATE TABLE transactions (
    transaction_id INT,
    user_id INT,
    transaction_date TEXT,
    amount REAL,
    platform TEXT
);
CREATE TABLE user_preferences (
    user_id INT,
    communication_preference TEXT,
    interface_theme TEXT,
    notification_settings TEXT
);
''')

# 3. Insert adjusted mock data
cursor.executemany('INSERT INTO user_activity VALUES (?, ?, ?, ?)', [
    (1, '2025-07-20', 10, 'Premium'),
    (2, '2025-07-18', 5, 'Basic'),
    (3, '2025-07-15', 3, 'Premium')
])
cursor.executemany('INSERT INTO transactions VALUES (?, ?, ?, ?, ?)', [
    (1, 1, '2025-07-01', 120.50, 'Web'),
    (2, 1, '2025-07-05', 300.00, 'Mobile'),
    (3, 2, '2025-07-10', 250.00, 'Web'),
    (4, 3, '2025-07-12', 400.00, 'Web')
])
cursor.executemany('INSERT INTO user_preferences VALUES (?, ?, ?, ?)', [
    (1, 'Email', 'Dark', 'Enabled'),
    (2, 'SMS', 'Light', 'Disabled'),
    (3, 'Push', 'Dark', 'Enabled')
])

# 4. SQL Query (enhanced)
query = f'''
WITH active_users AS (
    SELECT user_id
    FROM user_activity
    WHERE DATE(last_login_date) >= DATE('now', '-{days_active} day')
),
user_spending AS (
    SELECT user_id,
           SUM(amount) AS total_spent,
           AVG(amount) AS avg_transaction_value,
           COUNT(transaction_id) AS transaction_count
    FROM transactions
    GROUP BY user_id
),
spending_threshold AS (
    SELECT MAX(total_spent) * 0.8 AS top_20_threshold
    FROM user_spending
),
high_value_users AS (
    SELECT us.user_id, us.total_spent, us.avg_transaction_value, us.transaction_count
    FROM user_spending us
    CROSS JOIN spending_threshold st
    WHERE us.total_spent >= st.top_20_threshold
)
SELECT ua.user_id,
       ua.account_type,
       hv.total_spent,
       hv.avg_transaction_value,
       hv.transaction_count,
       up.communication_preference,
       up.interface_theme,
       up.notification_settings
FROM active_users a
JOIN high_value_users hv ON a.user_id = hv.user_id
JOIN user_activity ua ON a.user_id = ua.user_id
JOIN user_preferences up ON a.user_id = up.user_id;
'''

# 5. Run query and show results
df = pd.read_sql_query(query, conn)
print(f"Segmented Customers (Active in last {days_active} days):\n", df)


Segmented Customers (Active in last 30 days):
    user_id account_type  total_spent  avg_transaction_value  \
0        1      Premium        420.5                 210.25   
1        3      Premium        400.0                 400.00   

   transaction_count communication_preference interface_theme  \
0                  2                    Email            Dark   
1                  1                     Push            Dark   

  notification_settings  
0               Enabled  
1               Enabled  


# Scenario Questions

1.   Instead of using PERCENTILE_CONT (not available in SQLite), I calculated a dynamic threshold based on 80% of the highest total spending. This approach identifies top spenders even in small datasets.
2.   The query uses DATE(last_login_date) >= DATE('now', '-X day'), where X is a parameter (days_active). This makes the active user window flexible instead of fixed at 30 days.
3.Common Table Expressions (CTEs) break the logic into steps: identifying active users, calculating spending and thresholds, and selecting high-value users. This improves readability and allows reusing intermediate results without repeating logic.

